<a href="https://colab.research.google.com/github/daniel-hain/bibliometrics_EIST_2021/blob/master/python/BERTopic_EIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTopic EIST 

* First tryout on EIST topic modelling using BERT an reprentation learning.


## Setup

In [ ]:
# Installing BERTopic
!pip install bertopic

1.  **Enabling the GPU.** Navigate to Edit→Notebook Settings -> select GPU from the Hardware Accelerator drop-down
2. **Install BERTTopic:** Will need to be done
3. **Restart the Notebook:**Runtime → Restart Runtime

## Data

* Abstracts of all EIST articles up to XX.2023

In [1]:
import pandas as pd

In [2]:
docs = pd.read_csv("https://raw.githubusercontent.com/daniel-hain/bibliometrics_EIST_2021/master/data/data_text.csv")

In [4]:
docs.head()

,XX,text
0,GEELS_FW_2011_THE_MULTI_LEVEL_PERSPECTI,the multi-level perspective (mlp) has emerged ...
1,FRENKEN_K_2017_PUTTING_THE_SHARING_ECONO,we develop a conceptual framework that allows ...
2,KHLER_J_2019_AN_AGENDA_FOR_SUSTAINABIL,research on sustainability transitions has exp...
3,HANSEN_T_2015_THE_GEOGRAPHY_OF_SUSTAINA,this review covers the recent literature on th...
4,MEADOWCROFT_J_2011_ENGAGING_WITH_THE_POLITIC,although recent scholarship has contributed to...


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

* We start by instantiating BERTopic. 
*We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 
* We will also calculate the topic probabilities. 


In [83]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import KeyBERTInspired
from bertopic.representation import TextGeneration

from sentence_transformers import SentenceTransformer
from transformers.pipelines import pipeline

from hdbscan import HDBSCAN
from umap import UMAP

from sklearn.feature_extraction.text import CountVectorizer

In [103]:
# EMBEDDING MODEL:
embedding_model = "allenai-specter"

# DIMENSIONALITY REDUCTION
umap_model = UMAP(n_neighbors=5, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=1337)

# CLUSTERING: resuce min_cluster_size and min_samples
hdbscan_model = HDBSCAN(min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        prediction_data=True, 
                        min_samples=2)

# VECTORIZER: custom vectorizer to get rid of stopwords
vectorizer_model = CountVectorizer(stop_words="english",
                                   min_df = 10)

# TFIDF Weighting: Weighting model with redued stoppwords
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True,
                                     reduce_frequent_words=True)

# REPRESENTATION MODEL: Create your representation model
representation_model = KeyBERTInspired()

#prompt = "I have a topic described by the following keywords: [KEYWORDS]. Based on the previous keywords, what is this topic about?""
#generator = pipeline('text2text-generation', model='google/flan-t5-base')
#representation_model = TextGeneration(generator)

# BEWRTopic: specify all custom models and n_grams
topic_model = BERTopic(language="english", 
                       calculate_probabilities=True,
                       verbose=True, 
                       n_gram_range=(1, 3),
                       #
                       embedding_model=embedding_model, 
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model, 
                       vectorizer_model=vectorizer_model,
                       ctfidf_model=ctfidf_model,
                       representation_model=representation_model)

In [104]:
topics, probs = topic_model.fit_transform(docs.loc[:,'text'])

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

2023-02-16 09:16:44,278 - BERTopic - Transformed documents to Embeddings
2023-02-16 09:16:45,840 - BERTopic - Reduced dimensionality
2023-02-16 09:16:45,907 - BERTopic - Clustered reduced embeddings


In [105]:
# Calculate Hirarchy
hierarchical_topics = topic_model.hierarchical_topics(docs.loc[:,'text'])

100%|██████████| 36/36 [00:04<00:00,  7.25it/s]


In [106]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs.loc[:,'text'], calculate_tokens=True)

100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


In [ ]:
# Calculate topics over time
topics_over_time = topic_model.topics_over_time(docs.loc[:,'text'],
                                                docs.loc[:,'PY'])

## Extracting Topics
* After fitting our model, we can start by looking at the results. 
* Typically, we look at the most frequent topics first as they best represent the collection of documents. 
* -1 refers to all outliers and should typically be ignored. 

In [107]:
freq = topic_model.get_topic_info(); freq.head(50)

,Topic,Count,Name
0,-1,40,-1_government_governments_initiatives_governance
1,0,50,0_innovations_transformative_emissions_technol...
2,1,30,1_electricity_renewable_government_energy
3,2,24,2_sustainability_transformative_frameworks_gov...
4,3,20,3_government_governance_politics_scholars
5,4,17,4_economy_economic_sustainable_investment
6,5,17,5_sociotechnical_actors_scholars_change
7,6,16,6_innovation_innovations_industries_technological
8,7,16,7_sustainable_urban_infrastructure_framework
9,8,15,8_niches_sociotechnical_environmental_studies


## Attributes

* There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [108]:
topic_model.topics_[:10]

[2, 14, 2, 18, 34, -1, 8, 14, 18, 21]

# **Visualization**
* There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. 
* Topic modeling is, to a certain extent, quite subjective. * Visualizations help understand the topics that were created. 

## General topic relation

In [109]:
topic_model.visualize_topics()

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [110]:
topic_model.visualize_barchart(top_n_topics=50, n_words = 15)

## Visualize Topic Hierarchy

* The topics that were created can be hierarchically reduced. 
* In order to understand the potential hierarchical structure of the topics, we can create clusters and visualize how they relate to one another. 
* This might also help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [111]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

## Topics over time

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [112]:
topic_model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

## Visualize Topic Probabilities

- The variable `probabilities` that is returned from `transform()` or `fit_transform()` can  be used to understand how confident BERTopic is that certain topics can be found in a document. 



In [113]:
# Select a document
doc_nr = 102
docs.loc[doc_nr,'text']

"local level decision-making can provide a vital and practicable means through which to deliver carbon reductions at individual, household and community levels. this paper explores the opportunities and challenges associated with the role of local government action in delivering a resilient, low carbon future. drawing on qualitative data gathered through interviews with six case study local authorities in the uk, the paper examines in particular the ways in which they have tried to address energy and environmental goals in their areas of jurisdiction. the principles of multi-level perspective (mlp) are employed to provide fresh insights on the potential for local government to encourage 'niche' level influences on economic, technological and social/behavioural innovation. in doing so, we identify the enabling factors and drivers that have given rise to success in linking microlevel goals with existing macro-level objectives, as well as highlighting some of the persistent obstacles that

In [114]:
# Get the probability distribution
topic_model.visualize_distribution(probs[doc_nr], min_probability=0.01)

In [115]:
# Visualize the token-level distributions
doc_topic_dist = topic_model.visualize_approximate_distribution(docs.loc[doc_nr,'text'], topic_token_distr[doc_nr])
doc_topic_dist